In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q torchaudio transformers datasets soundfile
!apt-get install -y -qq ffmpeg
!pip install gtts
!pip install torch torchaudio transformers librosa soundfile
!apt-get install -y ffmpeg
!pip install sacremoses
!pip install scikit-learn
!pip install pythainlp
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.0
    Uninstalling click-8.3.0:
      Successfully uninstalled click-8.3.0
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 23.4 MB/s eta

In [ ]:
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 26.7 MB/s eta 0:00:00


In [ ]:
import os
import csv
import numpy as np
import torch
from gtts import gTTS
from pythainlp.tokenize import word_tokenize
from transformers import pipeline
from tqdm import tqdm
import pandas as pd
from pydub import AudioSegment
import torchaudio
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from difflib import SequenceMatcher
from pythainlp import romanize
import warnings

warnings.filterwarnings("ignore")
from transformers.utils import logging
logging.set_verbosity_error()

# ==========================================
# Configurazioni
# ==========================================
CSV_INPUT = "/content/drive/MyDrive/TesiMaggistrale/testeset_with_counts.csv"
CARTELLA_OUTPUT = "/content/drive/MyDrive/TesiMaggistrale/AudiDerivatiCorretti/TestRQ3/"
CSV_OUTPUT = os.path.join(CARTELLA_OUTPUT, "risultati.csv")
CSV_ERRORS = os.path.join(CARTELLA_OUTPUT, "errori_log.csv")

os.makedirs(CARTELLA_OUTPUT, exist_ok=True)

device_num = 0 if torch.cuda.is_available() else -1
print(f"Device in uso: {'GPU' if device_num==0 else 'CPU'}")

# ==========================================
# Modello Similarità a livello di frase
# ==========================================
sem_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

def semantic_similarity(text1, text2):
    if not text1 or not text2:
        return 0.0
    emb1 = sem_model.encode([text1])[0]
    emb2 = sem_model.encode([text2])[0]
    score = cosine_similarity([emb1], [emb2])[0][0]
    return float(score)

# ==========================================
# Calcolo gravità testuale basata SOLO sul numero di errori
# ==========================================
def calcola_gravita_testuale(numero_errori: int) -> tuple:
    """
    Calcola la gravità SOLO in base al numero di errori.
    Ritorna (gravita, livello_descrittivo)
    """
    gravita = float(numero_errori)

    if gravita == 0:
        livello = "Nessun errore"
    elif gravita <= 3:
        livello = "Lieve"
    elif gravita <= 6:
        livello = "Media"
    else:
        livello = "Grave"

    return gravita, livello

# ==========================================
# Generatore feedback aggiornato
# ==========================================
def genera_feedback(sim_score: float, livello_errore: str, frase_errata: str, frase_corretta: str) -> str:
    suggerimenti = analizza_differenze_fonetiche(frase_errata, frase_corretta)
    if livello_errore == "Nessun errore significativo":
        base_feedback = f"La frase '{frase_errata}' è semanticamente corretta ({sim_score:.2f}). Ottimo lavoro!"
    elif livello_errore == "Lieve":
        base_feedback = f"La frase '{frase_errata}' è molto simile a '{frase_corretta}' ({sim_score:.2f}). Solo piccole variazioni, ma il significato resta chiaro."
    elif livello_errore == "Media":
        base_feedback = f"La frase '{frase_errata}' differisce moderatamente da '{frase_corretta}' ({sim_score:.2f}). È comprensibile, ma può essere migliorata per una pronuncia più naturale."
    else:
        base_feedback = f"La frase '{frase_errata}' risulta piuttosto diversa da '{frase_corretta}' ({sim_score:.2f}). Il significato potrebbe cambiare: conviene ripetere con attenzione."
    if suggerimenti:
        dettagli = "\nSuggerimenti:\n- " + "\n- ".join(suggerimenti)
        return base_feedback + dettagli
    return base_feedback

# ==========================================
# Funzioni audio
# ==========================================
def prepara_audio(percorso_audio):
    ext = percorso_audio.split(".")[-1].lower()
    if ext == "mp3":
        audio = AudioSegment.from_file(percorso_audio)
        audio = audio.set_channels(1).set_frame_rate(16000)
        return np.array(audio.get_array_of_samples()).astype(np.float32) / 32768.0
    elif ext == "wav":
        waveform, sample_rate = torchaudio.load(percorso_audio)
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)
        if sample_rate != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
            waveform = resampler(waveform)
        return waveform.squeeze().numpy().astype(np.float32)
    else:
        raise ValueError(f"Formato non supportato: {ext}")

# ==========================================
# Tono e differenze fonetiche
# ==========================================
TONI = {"\u0E48": "alto", "\u0E49": "basso", "\u0E4A": "discendente", "\u0E4B": "ascendente"}

def rileva_tono(parola: str) -> str:
    for c in parola:
        if c in TONI:
            return TONI[c]
    return "neutro"

def analizza_differenze_fonetiche(frase_errata, frase_corretta):
    suggerimenti = []
    parole_err = word_tokenize(frase_errata, keep_whitespace=False)
    parole_cor = word_tokenize(frase_corretta, keep_whitespace=False)
    max_len = max(len(parole_err), len(parole_cor))
    for i in range(max_len):
        if i >= len(parole_err):
            suggerimenti.append(f"Hai omesso la parola '{parole_cor[i]}'")
            continue
        if i >= len(parole_cor):
            suggerimenti.append(f"Hai aggiunto una parola in più: '{parole_err[i]}'")
            continue
        parola_stud, parola_corr = parole_err[i], parole_cor[i]
        if parola_stud == parola_corr:
            continue
        tono_stud, tono_corr = rileva_tono(parola_stud), rileva_tono(parola_corr)
        if tono_stud != tono_corr:
            suggerimenti.append(f"Hai sbagliato il tono nella parola '{parola_stud}': da {tono_stud} → {tono_corr}")
        rom_stud, rom_corr = romanize(parola_stud), romanize(parola_corr)
        if SequenceMatcher(None, rom_stud, rom_corr).ratio() < 0.75:
            suggerimenti.append(f"Migliora la pronuncia di '{parola_stud}', dovrebbe essere '{parola_corr}'")
    return suggerimenti

# ==========================================
# Pipeline ASR
# ==========================================
MODEL_PATH = "/content/drive/MyDrive/TesiMaggistrale/audiErrati/wav2vec2_finetuned_th_erroriTotal"
pipe_asr = pipeline("automatic-speech-recognition", model=MODEL_PATH,
                    tokenizer=MODEL_PATH, feature_extractor=MODEL_PATH,
                    device=device_num)
pipe_asr_ref = pipeline("automatic-speech-recognition", model="airesearch/wav2vec2-large-xlsr-53-th")

# ==========================================
# Elaborazione batch dal CSV
# ==========================================
df_input = pd.read_csv(CSV_INPUT)
audio_col = "audio_path"

fieldnames = ["file", "trascrizione_errata", "trascrizione_corretta",
              "semantic_similarity", "gravita", "gravita_testuale",
              "feedback_semantico"]

with open(CSV_ERRORS, mode="w", newline="", encoding="utf-8") as ferr:
    csv.DictWriter(ferr, fieldnames=["file", "reason"]).writeheader()

with open(CSV_OUTPUT, mode="w", newline="", encoding="utf-8") as fcsv, \
     open(CSV_ERRORS, mode="a", newline="", encoding="utf-8") as ferr_append:

    writer = csv.DictWriter(fcsv, fieldnames=fieldnames)
    writer.writeheader()
    err_writer = csv.DictWriter(ferr_append, fieldnames=["file", "reason"])

    for _, row in tqdm(df_input.iterrows(), total=len(df_input), desc="Elaborazione CSV"):
        percorso_audio = row[audio_col]
        audio_out = os.path.join(CARTELLA_OUTPUT, os.path.splitext(os.path.basename(percorso_audio))[0] + ".mp3")

        try:
            audio = prepara_audio(percorso_audio)
            if audio is None or len(audio) == 0:
                err_writer.writerow({"file": percorso_audio, "reason": "audio vuoto/errore caricamento"})
                continue

            trascrizione = pipe_asr({"array": audio, "sampling_rate": 16000}).get("text", "").strip()
            if not trascrizione:
                err_writer.writerow({"file": percorso_audio, "reason": "trascrizione ASR vuota"})
                continue

            tts = gTTS(text=trascrizione, lang='th')
            tts.save(audio_out)

            audio_corr = prepara_audio(audio_out)
            trascrizione_corr = pipe_asr_ref({"array": audio_corr, "sampling_rate": 16000}).get("text", "").strip()

            sim_score = semantic_similarity(trascrizione, trascrizione_corr)

            # ==========================================
            # Gravità calcolata SOLO in base al numero di errori
            # ==========================================
            numero_errori = int(row.get("TOTALE_errori", 0))
            gravita, gravita_testuale = calcola_gravita_testuale(numero_errori)

            feedback = genera_feedback(sim_score, gravita_testuale, trascrizione, trascrizione_corr)

            writer.writerow({
                "file": percorso_audio,
                "trascrizione_errata": trascrizione,
                "trascrizione_corretta": trascrizione_corr,
                "semantic_similarity": f"{sim_score:.4f}",
                "gravita": f"{gravita:.4f}",
                "gravita_testuale": gravita_testuale,
                "feedback_semantico": feedback
            })

        except Exception as e:
            err_writer.writerow({"file": percorso_audio, "reason": str(e)})


Device in uso: CPU



Elaborazione CSV: 100%|██████████| 326/326 [56:20<00:00, 10.37s/it]
